In [1]:
import os
import sys

project_path = os.path.abspath(os.path.join(os.getcwd(),'..','..','..'))
sys.path.append(project_path)
print(f"Project Path: {project_path}")

Project Path: c:\Users\dalej\Documents\_Coding\DragonRegen


In [2]:
from src.AIGuardian.Tasks.KafkaAgent import KafkaAgent

agent = KafkaAgent(end_processing=20)
agent.receive_messages()

Agent Loop Count: 0 and last consumed time: 2025-05-22 12:19:22.045846
Agent Loop Count: 1 and last consumed time: 2025-05-22 12:19:22.045846
==> Running DataColumnType with prompt: None
==> Task 651a38fa-df9f-4abb-b47e-be0465965ed6 complete_task
==> Running DataColumnRefiner with prompt: None
==> Task faac7f1f-7688-499b-8bb3-d1e8a35759c8 complete_task
==> Running DataColumnType with prompt: None
==> Task 3d79e09f-76ae-444e-8958-92dfc6e290c7 complete_task
Agent Loop Count: 2 and last consumed time: 2025-05-22 12:19:34.044595
==> Running DataColumnType with prompt: None
==> Task 050ef18d-0eb4-4896-bebc-4eece2f91fbd complete_task
==> Running DataColumnRefiner with prompt: None
==> Task 7530bd6e-0dd4-42ea-ae0e-e938fe7a55f3 complete_task
==> Running DataColumnType with prompt: None
==> Task 7049997b-4c0d-4a82-b575-35e1debfa2fd complete_task
Agent Loop Count: 3 and last consumed time: 2025-05-22 12:19:40.430711
==> Running DataColumnType with prompt: None
==> Task c87afab5-6204-46ce-8b1e-d5

In [3]:
print(agent.waiting_tasks)
print(agent.processed_tasks)

[<src.AIGuardian.Tasks.ColumnRefiner.ColumnRefiner object at 0x000001A655FF8510>, <src.AIGuardian.Tasks.ColumnRefiner.ColumnRefiner object at 0x000001A65607A910>, <src.AIGuardian.Tasks.ColumnRefiner.ColumnRefiner object at 0x000001A65809CD90>]
[<src.AIGuardian.Tasks.DataColumnType.DataColumnType object at 0x000001A6546FF190>, <src.AIGuardian.Tasks.DataColumnRefiner.DataColumnRefiner object at 0x000001A6544F0250>, <src.AIGuardian.Tasks.DataColumnType.DataColumnType object at 0x000001A655EBE550>, <src.AIGuardian.Tasks.DataColumnType.DataColumnType object at 0x000001A655F772D0>, <src.AIGuardian.Tasks.DataColumnRefiner.DataColumnRefiner object at 0x000001A655FC8A10>, <src.AIGuardian.Tasks.DataColumnType.DataColumnType object at 0x000001A655EBDF90>, <src.AIGuardian.Tasks.DataColumnType.DataColumnType object at 0x000001A655FC8E10>, <src.AIGuardian.Tasks.DataColumnRefiner.DataColumnRefiner object at 0x000001A655FD94D0>, <src.AIGuardian.Tasks.DataColumnRefiner.DataColumnRefiner object at 0x000

---
## Debug the Agent not fully waiting

In [17]:
from src.MetaFort.AILoggingTopics import AILoggingTopics
print(type(agent.waiting_tasks[0]))
print(agent.waiting_tasks[0].child_task)
print(agent.db_engine.consumers[AILoggingTopics.AI_TASK_COMPLETED_TOPIC].config['group_id'])
print(agent.waiting_tasks[0].is_waiting())


<class 'src.AIGuardian.Tasks.ColumnRefiner.ColumnRefiner'>
['116ada54-1e14-40e0-b3a8-89181825d61d']
task-completed-ab13263c-6415-4ff8-9819-04557ec99c9c
True


In [15]:
print(agent.waiting_tasks)
print(agent.received_tasks)
print(agent.processed_tasks)
print(agent.completed_tasks)
print(agent.completed_tasks_received)
print(agent.completed_tasks_received_d1)

[<src.AIGuardian.Tasks.ColumnRefiner.ColumnRefiner object at 0x000001A65607A910>, <src.AIGuardian.Tasks.ColumnRefiner.ColumnRefiner object at 0x000001A65809CD90>]
[<src.AIGuardian.Tasks.DataColumnType.DataColumnType object at 0x000001A6546FF190>, <src.AIGuardian.Tasks.DataColumnRefiner.DataColumnRefiner object at 0x000001A6544F0250>, <src.AIGuardian.Tasks.DataColumnType.DataColumnType object at 0x000001A655EBE550>, <src.AIGuardian.Tasks.DataColumnType.DataColumnType object at 0x000001A655F772D0>, <src.AIGuardian.Tasks.DataColumnRefiner.DataColumnRefiner object at 0x000001A655FC8A10>, <src.AIGuardian.Tasks.DataColumnType.DataColumnType object at 0x000001A655EBDF90>, <src.AIGuardian.Tasks.DataColumnType.DataColumnType object at 0x000001A655FC8E10>, <src.AIGuardian.Tasks.DataColumnRefiner.DataColumnRefiner object at 0x000001A655FD94D0>, <src.AIGuardian.Tasks.DataColumnRefiner.DataColumnRefiner object at 0x000001A655FC8850>, <src.AIGuardian.Tasks.DataColumnRefiner.DataColumnRefiner object 

---
Looking into issues with kafka missing waits

In [18]:
ls_waiting_tasks = []
ls_waiting_tasks_child = {}
for item in agent.waiting_tasks:
    if item.is_waiting():
        ls_waiting_tasks.append(item.task_id)
        ls_waiting_tasks_child[item.task_id] = item.child_task
print(len(ls_waiting_tasks))
print(len(ls_waiting_tasks_child))


2
2


In [19]:
ls_received_tasks = []
for item in agent.received_tasks:
    ls_received_tasks.append(item.task_id)
print(len(ls_received_tasks))

103


In [20]:
ls_processed_tasks = []
for item in agent.processed_tasks:
    ls_processed_tasks.append(item.task_id)
print(len(ls_processed_tasks))

103


In [21]:
ls_completed_tasks = []
for item in agent.completed_tasks:
    ls_completed_tasks.append(item.task_id)
print(len(ls_completed_tasks))

100


## Investigate with searchs

In [22]:
print(ls_waiting_tasks_child)
print(agent.start_time)


{'18880711-6e9c-4dea-a0fa-9ec8aaca1936': ['116ada54-1e14-40e0-b3a8-89181825d61d'], '8a50c495-b8ea-4cf9-8fd1-df7964f30fcf': ['23ba0ea9-1208-459b-8b9d-c0df0944eb62']}
2025-05-22 12:19:22.044846


In [29]:
# search_task_id = '18880711-6e9c-4dea-a0fa-9ec8aaca1936'
search_task_id = '116ada54-1e14-40e0-b3a8-89181825d61d'
if search_task_id in ls_processed_tasks:
    print("processed tasks")
if search_task_id in ls_received_tasks:
    print("received tasks")
if search_task_id in ls_completed_tasks:
    print("Task is in completed tasks")
if search_task_id in agent.completed_tasks_received:
    print("Task is in received completed tasks")

processed tasks
received tasks
Task is in completed tasks
Task is in received completed tasks


In [30]:
print(agent.completed_tasks_received_d1['18880711-6e9c-4dea-a0fa-9ec8aaca1936'])
if search_task_id in agent.completed_tasks_received_d1['18880711-6e9c-4dea-a0fa-9ec8aaca1936']:
    print("Task is in completed tasks received d1")

['aaa89b33-6e5a-4a1c-80f9-40c8c0f3eb14', '23138e8f-0c4b-4d4a-9d38-5a75248287fd', '6ee14855-bd00-476d-87eb-8d040cdba3c4', '44d6a0eb-449b-408c-a992-cbe8b95c151d', 'd6bae60f-65c4-459a-9e53-e0bfc303318c', 'ea94d016-ca4b-47da-8007-dd18fd44fd2b', '6f82fe7c-a4dc-42fe-be8c-4b253e90f305', 'ac797d46-f2b0-4584-84d0-92d9aaa5443e', 'b7ef05a0-ae46-47ea-af87-362c171634a3', 'c6cd1372-063c-4e78-8d46-cecab2d21031', '798e15f1-1fab-40df-a092-459514f1cb44', '4eb9ac4e-1d17-4650-bc7f-40b3a69d4164', 'acdc02c6-fbda-4870-9262-08eaa93a6fb8', '06165a65-31cc-46e1-8381-e8e55a478ec5', '28a9a657-0a59-4614-a812-dd54063d9935', '9c892089-7fe8-4614-b70d-1a57a2bb9434', '207f93a8-aa11-4677-bb3a-6fbbf5940b4b', '066484b7-ccbb-4d31-a4d5-b0f92628fe79', 'b6c8fe98-c939-4cb7-b1a2-760aa0b77d60', '23184e18-1414-4379-b208-b46efebf7d02', 'bd8ab7f7-fc66-46c0-9bbf-12b9bf604893', '947654ee-c8af-4ba7-9168-d924c2fdfffd', 'fe195c27-9b44-492b-a33b-b6573ab19cd5', '049ce88f-0c3d-411f-afe8-d7e8e092e7f3', 'fc42e7f3-1a7c-4df8-ae89-379f0b8acd16',

In [ ]:
print(agent.waiting_tasks[0].task_id)
# print(agent.waiting_tasks[0].processed_comp_tasks)
# print(agent.waiting_tasks[0].removed_comp_tasks)
if search_task_id in agent.waiting_tasks[0].processed_comp_tasks:
    print("Boom processsed!")
if search_task_id in agent.waiting_tasks[0].removed_comp_tasks:
    print("Boom!")

18880711-6e9c-4dea-a0fa-9ec8aaca1936
['aaa89b33-6e5a-4a1c-80f9-40c8c0f3eb14', '23138e8f-0c4b-4d4a-9d38-5a75248287fd', '6ee14855-bd00-476d-87eb-8d040cdba3c4', '44d6a0eb-449b-408c-a992-cbe8b95c151d', 'd6bae60f-65c4-459a-9e53-e0bfc303318c', 'ea94d016-ca4b-47da-8007-dd18fd44fd2b', '6f82fe7c-a4dc-42fe-be8c-4b253e90f305', 'ac797d46-f2b0-4584-84d0-92d9aaa5443e', 'b7ef05a0-ae46-47ea-af87-362c171634a3', 'c6cd1372-063c-4e78-8d46-cecab2d21031', '798e15f1-1fab-40df-a092-459514f1cb44', '4eb9ac4e-1d17-4650-bc7f-40b3a69d4164', 'acdc02c6-fbda-4870-9262-08eaa93a6fb8', '06165a65-31cc-46e1-8381-e8e55a478ec5', '28a9a657-0a59-4614-a812-dd54063d9935', '9c892089-7fe8-4614-b70d-1a57a2bb9434', '207f93a8-aa11-4677-bb3a-6fbbf5940b4b', '066484b7-ccbb-4d31-a4d5-b0f92628fe79', 'b6c8fe98-c939-4cb7-b1a2-760aa0b77d60', '23184e18-1414-4379-b208-b46efebf7d02', 'bd8ab7f7-fc66-46c0-9bbf-12b9bf604893', '947654ee-c8af-4ba7-9168-d924c2fdfffd', 'fe195c27-9b44-492b-a33b-b6573ab19cd5', '049ce88f-0c3d-411f-afe8-d7e8e092e7f3', 'f

In [13]:
from src.MetaFort.SysLogs.KafkaEngine import KafkaEngine
kafka_engine = KafkaEngine('localhost:9092')

print("==> Reviewing waiting tasks")
for task in agent.waiting_tasks:
    print(f"==> Reviewing task {task.task_id}")
    for child_task in task.child_task:
        print(f"=====> Reviewing child task {child_task}")
        found_msg = kafka_engine.search_batch_topic(AILoggingTopics.AI_TASK_COMPLETED_TOPIC, search_value=child_task, search_key='task_id')
        if found_msg:
            print(f"=====> Found message for child task {child_task}")
            task.child_task.remove(child_task)
    if not task.is_waiting():
        print('BOOM!!!')
        agent.waiting_tasks.remove(task)

==> Reviewing waiting tasks
==> Reviewing task b0822840-c340-4254-92a6-849cdc888ba4
=====> Reviewing child task d1dca9e1-c57b-4c61-acbc-8c167b4feade
Searching topic 'ai_tasks_completed' for value: d1dca9e1-c57b-4c61-acbc-8c167b4feade
This will timeout after 60 seconds if no more messages are available.
Processing 360 messages from partition TopicPartition(topic='ai_tasks_completed', partition=1)
Processing 323 messages from partition TopicPartition(topic='ai_tasks_completed', partition=2)
Processing 331 messages from partition TopicPartition(topic='ai_tasks_completed', partition=4)
Processing 330 messages from partition TopicPartition(topic='ai_tasks_completed', partition=3)
Processing 309 messages from partition TopicPartition(topic='ai_tasks_completed', partition=0)
Found match at offset 304 in partition 0


Search completed in 1.16 seconds.
Processed 1653 messages, found 1 matches.
=====> Found message for child task d1dca9e1-c57b-4c61-acbc-8c167b4feade
BOOM!!!
==> Reviewing task 8a50c495-b8ea-4cf9-8fd1-df7964f30fcf
=====> Reviewing child task 4eb63f0f-0901-4e57-a69d-419d970c03e1
Searching topic 'ai_tasks_completed' for value: 4eb63f0f-0901-4e57-a69d-419d970c03e1
This will timeout after 60 seconds if no more messages are available.
Processing 360 messages from partition TopicPartition(topic='ai_tasks_completed', partition=1)
Processing 309 messages from partition TopicPartition(topic='ai_tasks_completed', partition=0)
Processing 331 messages from partition TopicPartition(topic='ai_tasks_completed', partition=4)
Processed 1000 messages in 0.13 seconds...
Processed 1000 messages in 0.13 seconds...
Processed 1000 messages in 0.13 seconds...
Processed 1000 messages in 0.13 seconds...
Processed 1000 messages in 0.13 seconds...
Processed 1000 messages in 0.13 seconds...
Processed 1000 messages 